In [6]:
import numpy as np 
from pathlib import Path
from PIL import Image
from einops import rearrange
import re

In [7]:
root_path = Path.cwd()
fig_path = root_path / "fig"

pic_list = [path for path in fig_path.iterdir() if path.name[-3:] == "png"]
output_path = root_path / "split_fig"
if not output_path.exists():
    output_path.mkdir()

In [9]:
def set_table(a):
    table = []
    for i in range(256):
        if i < a:
            table.append(0)
        else:
            table.append(1)
    return table

def get_block_num(path):
    img = Image.open(path).resize((1024, 128), Image.Resampling.LANCZOS)
    tmp = np.array(img.point(set_table(50),'1') ).sum(axis=0) > 50
    feature = ''.join(str(list(tmp.astype(int))).strip('[]').split(', '))
    return len(re.findall('011111', feature))

def split_image(input_path, output_path, pic_name, h_split=2):
    w_split = get_block_num(input_path)
    img = Image.open(input_path)
    w, h = img.size
    pic = np.array(img.resize((w//w_split*w_split, h//h_split*h_split), Image.Resampling.LANCZOS))
    pic_split = rearrange(pic, "(d1 H) (d2 W) -> (d1 d2) H W",  d1 = h_split, d2 = w_split)
    for i, pic in enumerate(pic_split):
        tmp = Image.fromarray(pic)
        file_name = str(pic_name) + '_' + str(i) + '.png'
        tmp.save(output_path / file_name)

In [10]:
for i, path in enumerate(pic_list):
    split_image(path, output_path, i)
    print('finish picture: ', str(path.name))

finish picture:  2021_10_12__22_47_12.avi__1.png
finish picture:  2021_09_24__23_27_35.png
finish picture:  2021_10_12__22_44_58.avi__1.png
finish picture:  2021_09_24__21_32_11.png
finish picture:  2021_10_12__22_46_23.avi__1.png
